# Imports et configuration initiale

In [ ]:
import os
from detectron2.config import get_cfg
from detectron2 import model_zoo
from detectron2.data.datasets import register_coco_instances
from detectron2.engine.defaults import DefaultTrainer
from detectron2.evaluation import COCOEvaluator, inference_on_dataset
from detectron2.data import build_detection_test_loader
import shutil
import torch
import pickle


Enregistrer ton dataset

In [ ]:
# Enregistrement des jeux de données COCO personnalisés (train, valid, test)
register_coco_instances(
    "web_scrapper_images_annotes.v2i.coco/train", {},
    "web_scrapper_images_annotes.v2i.coco/train/_annotations.coco.json",
    "web_scrapper_images_annotes.v2i.coco/train"
)
register_coco_instances(
    "web_scrapper_images_annotes.v2i.coco/valid", {},
    "web_scrapper_images_annotes.v2i.coco/valid/_annotations.coco.json",
    "web_scrapper_images_annotes.v2i.coco/valid"
)
register_coco_instances(
    "web_scrapper_images_annotes.v2i.coco/test", {},
    "web_scrapper_images_annotes.v2i.coco/test/_annotations.coco.json",
    "web_scrapper_images_annotes.v2i.coco/test"
)


Configurer le modèle Faster R-CNN

In [ ]:
cfg = get_cfg()
cfg.merge_from_file(model_zoo.get_config_file("COCO-Detection/faster_rcnn_R_50_DC5_3x.yaml"))

# Datasets
cfg.DATASETS.TRAIN = ("web_scrapper_images_annotes.v2i.coco/train",)
cfg.DATASETS.TEST = ("web_scrapper_images_annotes.v2i.coco/valid",)

# Modèle pré-entraîné
cfg.MODEL.WEIGHTS = model_zoo.get_checkpoint_url("COCO-Detection/faster_rcnn_R_50_DC5_3x.yaml")
cfg.MODEL.ROI_HEADS.NUM_CLASSES = 18
cfg.MODEL.DEVICE = "cpu"  # ou "cuda" si GPU

# Hyperparamètres
cfg.DATALOADER.NUM_WORKERS = 2
cfg.SOLVER.IMS_PER_BATCH = 2
cfg.SOLVER.BASE_LR = 0.00025
cfg.SOLVER.MAX_ITER = 1000
cfg.SOLVER.STEPS = []
cfg.MODEL.ROI_HEADS.BATCH_SIZE_PER_IMAGE = 64

# Dossier de sortie
cfg.OUTPUT_DIR = "./output_detectron_web"
os.makedirs(cfg.OUTPUT_DIR, exist_ok=True)


Entraînement du modèle

In [ ]:
cfg.MODEL.DEVICE = "cpu" # ou "cuda" si GPU
# Entraînement
trainer = DefaultTrainer(cfg) 
trainer.resume_or_load(resume=False)
trainer.train()



[04/09 20:00:04 d2.engine.defaults]: Model:
GeneralizedRCNN(
  (backbone): ResNet(
    (stem): BasicStem(
      (conv1): Conv2d(
        3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False
        (norm): FrozenBatchNorm2d(num_features=64, eps=1e-05)
      )
    )
    (res2): Sequential(
      (0): BottleneckBlock(
        (shortcut): Conv2d(
          64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False
          (norm): FrozenBatchNorm2d(num_features=256, eps=1e-05)
        )
        (conv1): Conv2d(
          64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False
          (norm): FrozenBatchNorm2d(num_features=64, eps=1e-05)
        )
        (conv2): Conv2d(
          64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False
          (norm): FrozenBatchNorm2d(num_features=64, eps=1e-05)
        )
        (conv3): Conv2d(
          64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False
          (norm): FrozenBatchNorm2d(num_features=256, eps=1e-05)
 

model_final_68d202.pkl: 663MB [25:54, 426kB/s]                                
Skip loading parameter 'roi_heads.box_predictor.cls_score.weight' to the model due to incompatible shapes: (81, 1024) in the checkpoint but (10, 1024) in the model! You might want to double check if this is expected.
Skip loading parameter 'roi_heads.box_predictor.cls_score.bias' to the model due to incompatible shapes: (81,) in the checkpoint but (10,) in the model! You might want to double check if this is expected.
Skip loading parameter 'roi_heads.box_predictor.bbox_pred.weight' to the model due to incompatible shapes: (320, 1024) in the checkpoint but (36, 1024) in the model! You might want to double check if this is expected.
Skip loading parameter 'roi_heads.box_predictor.bbox_pred.bias' to the model due to incompatible shapes: (320,) in the checkpoint but (36,) in the model! You might want to double check if this is expected.
Some model parameters or buffers are not found in the checkpoint:
roi_heads

[04/09 20:26:00 d2.engine.train_loop]: Starting training from iteration 0


c:\Users\DELL\anaconda3\envs\detectron2-env\lib\site-packages\torch\functional.py:504: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at ..\aten\src\ATen\native\TensorShape.cpp:3484.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]


[04/09 20:32:14 d2.utils.events]:  eta: 4:27:32  iter: 19  total_loss: 4.106  loss_cls: 2.223  loss_box_reg: 0.7293  loss_rpn_cls: 0.8696  loss_rpn_loc: 0.2047    time: 17.5281  last_time: 14.9907  data_time: 0.6975  last_data_time: 0.0066   lr: 4.9953e-06  
[04/09 20:37:44 d2.utils.events]:  eta: 4:03:03  iter: 39  total_loss: 3.678  loss_cls: 2.068  loss_box_reg: 0.7065  loss_rpn_cls: 0.7777  loss_rpn_loc: 0.1958    time: 16.8403  last_time: 22.7729  data_time: 0.0072  last_data_time: 0.0070   lr: 9.9902e-06  
[04/09 20:43:04 d2.utils.events]:  eta: 3:55:04  iter: 59  total_loss: 3.24  loss_cls: 1.851  loss_box_reg: 0.6818  loss_rpn_cls: 0.4919  loss_rpn_loc: 0.1863    time: 16.5463  last_time: 21.7572  data_time: 0.0067  last_data_time: 0.0064   lr: 1.4985e-05  
[04/09 20:49:03 d2.utils.events]:  eta: 3:56:35  iter: 79  total_loss: 2.766  loss_cls: 1.524  loss_box_reg: 0.7403  loss_rpn_cls: 0.3509  loss_rpn_loc: 0.1654    time: 16.8977  last_time: 15.5174  data_time: 0.0049  last_da

Évaluation sur le jeu de test

In [ ]:
evaluator = COCOEvaluator("web_scrapper_images_annotes.v2i.coco/test", cfg, False, output_dir=cfg.OUTPUT_DIR)
test_loader = build_detection_test_loader(cfg, "web_scrapper_images_annotes.v2i.coco/test")

print(" Évaluation en cours...")
inference_on_dataset(trainer.model, test_loader, evaluator)


sauvegarde du model fintuné

In [ ]:
# 1. Chemin du modèle final entraîné
trained_model_path = os.path.join(cfg.OUTPUT_DIR, "model_final.pth")

# Vérifie que le modèle existe
if os.path.exists(trained_model_path):
    print(f" Modèle entraîné sauvegardé ici : {trained_model_path}")
else:
    print(" Aucun modèle trouvé. Vérifie que l'entraînement est terminé avec succès.")

# 2. Sauvegarde de la configuration utilisée (au format .pkl)
with open(os.path.join(cfg.OUTPUT_DIR, "config.pkl"), "wb") as f:
    pickle.dump(cfg, f)
    print(" Configuration sauvegardée dans : config.pkl")

# 3. Sauvegarde des métadonnées associées aux datasets
metadata = MetadataCatalog.get(cfg.DATASETS.TRAIN[0])
with open(os.path.join(cfg.OUTPUT_DIR, "metadata.pkl"), "wb") as f:
    pickle.dump(metadata, f)
    print(" Metadata sauvegardées dans : metadata.pkl")